In [1]:
import sqlite3, time, json, queue, threading, sys, os, asyncio

with open('secret.json') as fr: sec = json.load(fr)
# sec = {'blub_ip':"192.168.x.x", 'plug_ip':"192.168.x.x", 'gateway_ip':"192.168.x.x"}

In [2]:
def initialize_database(): 
    con = sqlite3.connect(f'SHT40.{datetime.datetime.fromtimestamp(time.time()).strftime("%Y-%m-%dT%H:%M:%S")}.db')  
    cur = con.cursor() 
    cur.execute('CREATE TABLE IF NOT EXISTS log  (Temperature REAL, Humidity REAL, Photoresistor INTEGER, timestamp REAL);')
    con.commit() 
    con.close() 
#initialize_database()

In [3]:
def database(): 
    con = sqlite3.connect('SHT40.2021-06-25T18:20:22.db')  
    cur = con.cursor() 
    while True: 
        opt, foo = q_in.get() # q.get() = opt_foo
        bar = False           # opt is a placeholder 
        try:
            if opt == 'insert':
                cur.execute('INSERT INTO log VALUES (?, ?, ?, ?);', 
                            (foo['Temperature (C)'], foo['Humidity (% rH)'], foo['Photoresistor'], time.time(),))  
            elif opt == 'sql':
                cur.execute(foo['sql'])  
                bar = cur.fetchall()
            elif opt == 'commit': 
                con.commit() 
                bar = 'Database commited'
                print('Database commited')
            elif opt == 'close': 
                con.commit()
                con.close()
                bar = 'Database commited and closed'
                print('Database commited and closed')
        except Exception as e:
            print('sql_exception', e, opt, foo)
            pass
        q_out.put(bar) 
        q_out.join()
        q_in.task_done()

q_in, q_out = queue.Queue(), queue.Queue() 
threading.Thread(target=database, name='thread-1', daemon=True).start() 

def database_operator(opt_foo): 
    q_in.join()        
    q_in.put(opt_foo)  
    bar = q_out.get()  
    q_out.task_done()   
    return bar

In [4]:
sys.path.append(f"{os.environ['HOME']}/py3/lib/python3.8/site-packages/") 
import kasa 

class HUB:
    def __init__(self):
        self.light = kasa.SmartBulb(sec['blub_ip'])
        self.ac    = kasa.SmartPlug(sec['plug_ip'])
        self.light_flag = False
        self.ac_flag    = False  
        threading.Thread(target=self.updater).start() 
            
    def updater(self):
        while 1:
            asyncio.run(self.light.update())
            asyncio.run(self.ac.update())
            if self.light.is_on: 
                self.light_flag = True
            else:   
                self.light_flag = False
            if self.ac.is_on: 
                self.ac_flag = True
            else:  
                self.ac_flag = False
            time.sleep(180)
hub = HUB()

In [ ]:
import flask
app = flask.Flask(__name__) 

import logging
log = logging.getLogger('werkzeug')
log.setLevel(logging.ERROR)

temp_ceil  = 27.5
temp_floor = 25.5
lumi_ceil  = 400
lumi_floor = 200

@app.route("/", methods=['POST'])
def index():
    T = flask.request.json['Temperature (C)']
    P = flask.request.json['Photoresistor']
    
    if T > temp_ceil and hub.ac_flag == False: 
        asyncio.run(hub.ac.turn_on()) 
        hub.ac_flag = True
    elif T < temp_floor and hub.ac_flag == True:
        asyncio.run(hub.ac.turn_off()) 
        hub.ac_flag = False
        
    if P > lumi_ceil and hub.light_flag == True: 
        asyncio.run(hub.light.turn_off()) 
        hub.light_flag = False
    elif P < lumi_floor and hub.light_flag == False:
        asyncio.run(hub.light.turn_on())    
        hub.light_flag = True     
        
    database_operator(('insert', flask.request.json)) 
    return '' 

@app.route("/sql", methods=['POST'])
def sql():  
    return {'result':database_operator(('sql', flask.request.json)).__repr__() }

@app.route('/heartbeat', methods=['GET']) 
def heartbeat():
    database_operator(('commit', None))  
    return 'Cheers!'

app.run(sec['gateway_ip'],'4433') 
# database_operator(('sql', {'sql':'select * from log'})) 
database_operator(('close', {'sql':'select * from log'}))

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
